In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
data=pd.read_csv("fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
data.shape

(20800, 5)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
data = data.drop(['id'],axis=1)

In [7]:
data=data.fillna('')

In [8]:
data['content'] = data['title'] + '' + data['text'] # Combine 'title' and 'text' columns into 'content'
data = data.drop(['title', 'text'], axis=1)          # Drop 'title' and 'text' columns

In [9]:
data.head()

,author,label,content
0,Darrell Lucus,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,Daniel J. Flynn,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Consortiumnews.com,1,Why the Truth Might Get You FiredWhy the Truth...
3,Jessica Purkiss,1,15 Civilians Killed In Single US Airstrike Hav...
4,Howard Portnoy,1,Iranian woman jailed for fictional unpublished...


In [10]:
# Convert to lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [11]:
# Remove punctuation

data['content'] = data['content'].str.replace('[^\w\s]','')

C:\Users\jajal\AppData\Local\Temp\ipykernel_21664\3602057189.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace('[^\w\s]','')


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jajal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
!pip install textblob

  Obtaining dependency information for textblob from https://files.pythonhosted.org/packages/1e/d6/40aa5aead775582ea0cf35870e5a3f16fab4b967f1ad2debe675f673f923/textblob-0.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for nltk>=3.9 from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
    --------------------------------------- 10.2/624.3 kB ? eta -:--:--
    --------------------------------------- 10.2/624.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/624.3 kB 187.9 kB/s eta 0:00:04
   - ------------------------------------- 30.7/624.3 kB 187.9 kB/s eta 0:00:04
   - ------------------------------------- 30.7/624.3 kB 187.9 kB/s eta 0:00:04
   -- ------------------------------------ 41.0/624.3 kB 131.3 kB/s eta 0:

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jajal\AppData\Roaming\nltk_data...


True

In [16]:
# Do lemmatization
from nltk.stem import WordNetLemmatizer
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

0    house dem aide didnt even see comeys letter ja...
1    flynn hillary clinton big woman campus breitba...
2    truth might get firedwhy truth might get fired...
3    15 civilian killed single u airstrike identifi...
4    iranian woman jailed fictional unpublished sto...
Name: content, dtype: object

In [17]:
X=data['content']
y=data['label']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45,stratify=y)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14560,)
(6240,)
(14560,)
(6240,)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

tfidf_vect.fit(data['content'])

xtrain_tfidf = tfidf_vect.transform(X_train)
xtest_tfidf = tfidf_vect.transform(X_test)

# Model Building
1.Passive Aggressive Classifier

In [23]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf,y_train)
predictions=pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3116
           1       0.94      0.95      0.94      3124

    accuracy                           0.94      6240
   macro avg       0.94      0.94      0.94      6240
weighted avg       0.94      0.94      0.94      6240



In [25]:
print(metrics.confusion_matrix(y_test,predictions))

[[2919  197]
 [ 170 2954]]


# 2.MLP Classifier

In [27]:
from sklearn.neural_network import MLPClassifier
mlpclf=MLPClassifier(hidden_layer_sizes=(256,64,16),
                    activation='relu',
                    solver='adam')
mlpclf.fit(xtrain_tfidf,y_train)
predictions=mlpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      3116
           1       0.94      0.95      0.95      3124

    accuracy                           0.95      6240
   macro avg       0.95      0.95      0.95      6240
weighted avg       0.95      0.95      0.95      6240



In [28]:
print(metrics.confusion_matrix(y_test,predictions))

[[2932  184]
 [ 151 2973]]


In [29]:
import pickle
pickle.dump(mlpclf,open("fakenews1.pkl","wb"))